In [1]:
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import glob
import numpy as np
import os

In [2]:
final_df = pd.DataFrame()
file_list = glob.glob("../examples/outputs/**/*.csv", recursive=True)

In [3]:
name_list = file_list[0].split("/")

file_name = name_list[-1].split(".")[0]

In [4]:
name_list

['..',
 'examples',
 'outputs',
 '50x50_grid',
 '45079254_50x50_grid_to_50_with_20_sims.csv']

In [5]:
def add_full_csv(full_file_name, final_df):
    df = pd.read_csv(full_file_name)
    
    name_list = full_file_name.split("/")
    file_name = name_list[-1].split(".")[0]
    shapefile = name_list[3]
    
    new_rows = []

    max_dist = df['district'].max()

    for i, district_df in df.groupby('district'):
        if i == max_dist:
            continue

        district_df = df[df['district'] == i]
        i_max = district_df['b1_probs'].max()
        i_med = district_df["b1_probs"].median()

        nonzero_probs = district_df[district_df['b1_probs'] != 0]['b1_probs']
        i_min = nonzero_probs.min() if not nonzero_probs.empty else 0

        ratio_min = i_max / i_min if i_min != 0 else None
        ratio_med = i_max / i_med

        new_rows.append({"Shapefile": shapefile,
                         "File Name": file_name,
                         "Generation": i, 
                         "Max": i_max, 
                         "Min": i_min, 
                         "Median": i_med,
                         "Ratio Max to Min": ratio_min,
                         "Ratio Max to Median": ratio_med})



    new_df = pd.DataFrame(new_rows, columns=["Shapefile",
                                             "File Name",
                                             "Generation", 
                                             "Max", 
                                             "Min", 
                                             "Median",
                                             "Ratio Max to Min",
                                             "Ratio Max to Median"])
    
    
    final_df = pd.concat([final_df, new_df])
    
    return final_df
    

In [6]:
me_df = pd.DataFrame()

for name in file_list:
    me_df = add_full_csv(name, me_df)

os.mkdir("../examples/report", exist_ok=True)
    
me_df.to_csv("../examples/report/SMC_full_data_output_(max,min,ratios).csv", index=False);

In [7]:
# me_df = pd.read_csv("./report/SMC_full_data_output_(max,min,ratios).csv")

In [8]:
worst_max_min_ratios_by_run = pd.DataFrame()
for _, b in me_df.groupby("Shapefile"):
    worst_max_min_ratios_by_run = pd.concat([worst_max_min_ratios_by_run, b.nlargest(1,"Ratio Max to Min")])

worst_max_med_ratios_by_run = pd.DataFrame()
for _, b in me_df.groupby("Shapefile"):
    worst_max_med_ratios_by_run = pd.concat([worst_max_med_ratios_by_run, b.nlargest(1,"Ratio Max to Median")])

In [10]:
worst_max_min_ratios_by_run.to_csv("../examples/report/SMC_largest_Max:Min_ratios_by_run.csv", index=False)
worst_max_med_ratios_by_run.to_csv("../examples/report/SMC_largest_Max:Median_ratios_by_run.csv", index=False)

In [11]:
average_max_min_by_shapefile = me_df.groupby("Shapefile")["Ratio Max to Min"].mean().reset_index()
average_max_min_by_shapefile.rename(columns={'Ratio Max to Min': 'Max:Min Average'}, inplace=True)


average_max_med_by_shapefile = me_df.groupby("Shapefile")["Ratio Max to Median"].mean().reset_index()
average_max_med_by_shapefile.rename(columns={'Ratio Max to Median': 'Max:Median Average'}, inplace=True)


In [12]:
average_max_min_by_shapefile.to_csv("../examples/report/Average_Max:Min_by_shapefile(all_runs).csv", index=False)
average_max_med_by_shapefile.to_csv("../examples/report/Average_Max:Median_by_shapefile(all_runs).csv", index=False)